In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from langchain_core.documents import Document

In [11]:
doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')
processed_content = doc['Processed_Content']

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\admin-npatel237\AppData\Local\Temp\ipykernel_1652\1769151165.py:1: SyntaxWarning: invalid escape sequence '\T'
  doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')


In [12]:
doc.columns

Index(['Section', 'Url', 'Content', 'Metadata', 'Processed_Content',
       'Processed_Section', 'Processed_Metadata'],
      dtype='object')

In [13]:
embedding_model = HuggingFaceEmbeddings(model_name="nlpaueb/legal-bert-base-uncased")

No sentence-transformers model found with name nlpaueb/legal-bert-base-uncased. Creating a new one with mean pooling.
c:\Users\admin-npatel237\AppData\Local\anaconda3\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin-npatel237\.cache\huggingface\hub\models--nlpaueb--legal-bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your

In [14]:
result = embedding_model.embed_query(text=processed_content[67])

In [15]:
documents = [Document(page_content=row['Processed_Content'], metadata={'Section': row['Processed_Section'], 'Metadata': row['Processed_Metadata'],'Url': row['Url']}) for i, row in doc.iterrows()]

In [ ]:
embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])

In [19]:
data = {
    'Processed_Content': [doc.page_content for doc in documents],
    'Embedding': embeddings,  # Assuming embeddings were created in the same order
    'Processed_Section': [doc.metadata['Section'] for doc in documents],
    'Processed_Metadata': [doc.metadata['Metadata'] for doc in documents],
    'Url': [doc.metadata['Url'] for doc in documents]
}

# Convert to DataFrame
df_embeddings = pd.DataFrame(data)

In [20]:
df_embeddings.to_csv('embeddings_nlpaueb_lbbu.csv', index=False)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Generate query embedding
query_embedding = embedding_model.embed_query(text=processed_content[67])

# Convert to NumPy arrays
query_embedding = np.array(query_embedding).reshape(1, -1)
embeddings = np.array(embeddings)

# Compute cosine similarity
similarities = cosine_similarity(query_embedding, embeddings)

# Get the index of the most similar document
most_similar_index = np.argmax(similarities)

print("Most similar document index:", most_similar_index)


Most similar document index: 67
